<a href="https://colab.research.google.com/github/Joshi-Atharva/Sentiment-analysis/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')
# Mounting the drive to use the files and images
import os
import csv
os.chdir('/content/drive/MyDrive/Sentiment analysis/')
print(os.listdir())
# rom extract_csv import extract_csv, split
from extract_bin import extract_bin, split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['Sentiment_analysis.ipynb', 'extract_bin.py', '__pycache__', 'amazon_cells_labelled.txt']


In [16]:
file_path = 'amazon_cells_labelled.txt'
data = extract_bin(file_path)
inputs = data.inputs
labels = data.labels
# data = extract_bin("train_set_sentiment_final.txt")
# input300 = data.inputs
# labels300 = data.labels
# inputs.extend(input300)
# labels.extend(labels300)

split_object = split(inputs, labels, 3500)
train_inputs = split_object.train_set_inputs
train_labels = split_object.train_set_labels
test_inputs = split_object.test_set_inputs
test_labels = split_object.test_set_labels
print(train_inputs)
print(train_labels)

['So there is no way for me to plug it in here in the US unless I go by a converter.\t', 'Good case, Excellent value.\t', 'Great for the jawbone.\t', 'Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!\t', 'The mic is great.\t', 'I have to jiggle the plug to get it to line up right to get decent volume.\t', 'If you have several dozen or several hundred contacts, then imagine the fun of sending each of them one by one.\t', 'If you are Razr owner...you must have this!\t', 'Needless to say, I wasted my money.\t', 'What a waste of money and time!.\t', 'And the sound quality is great.\t', 'He was very impressed when going from the original battery to the extended battery.\t', 'If the two were seperated by a mere 5+ ft I started to notice excessive static and garbled sound from the headset.\t', 'Very good quality though\t', 'The design is very odd, as the ear "clip" is not very comfortable at all.\t', 'Highly recommend for any one who has a blue tooth phone.\t', 

In [17]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_inputs)
print(tokenizer.word_index)
tokenized_inputs = tokenizer.texts_to_sequences(train_inputs)
print(tokenized_inputs)


{'and': 1, 'the': 2, 'to': 3, 'is': 4, 'a': 5, 'can': 6, 'of': 7, 'i': 8, 'it': 9, 'in': 10, 'this': 11, 'for': 12, 'leads': 13, 'brings': 14, 'life': 15, 'day': 16, 'phone': 17, 'with': 18, 'every': 19, 'my': 20, 'make': 21, 'was': 22, 'new': 23, 'be': 24, 'trust': 25, 'from': 26, 'work': 27, 'your': 28, 'goals': 29, 'opportunities': 30, 'you': 31, 'good': 32, 'success': 33, 'more': 34, 'not': 35, 'being': 36, 'endless': 37, 'on': 38, 'very': 39, 'hard': 40, 'destroys': 41, 'without': 42, 'great': 43, 'all': 44, 'has': 45, 'relationships': 46, 'others': 47, 'world': 48, 'lasting': 49, 'after': 50, 'growth': 51, 'than': 52, 'that': 53, 'decision': 54, 'at': 55, 'criticism': 56, 'encouragement': 57, 'morale': 58, 'first': 59, 'happiness': 60, 'have': 61, 'fear': 62, 'meaningful': 63, 'people': 64, 'even': 65, 'potential': 66, 'time': 67, 'better': 68, 'painful': 69, 'over': 70, 'failure': 71, 'discouraging': 72, 'create': 73, 'difficult': 74, 'creates': 75, 'makes': 76, 'something': 77,

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_inputs = pad_sequences(tokenized_inputs, padding='post')
train_labels = tf.convert_to_tensor(train_labels)
print(padded_inputs)


[[ 194  361    4 ...    0    0    0]
 [  32  240  262 ...    0    0    0]
 [  43   12    2 ...    0    0    0]
 ...
 [4530  321   13 ...    0    0    0]
 [4531  238  250 ...    0    0    0]
 [4532  279   75 ...    0    0    0]]


In [19]:
X = padded_inputs
Y = train_labels
print(X.shape)
print(Y.shape)
print(Y)

(3500, 30)
(3500,)
tf.Tensor([0 1 1 ... 0 1 0], shape=(3500,), dtype=int32)


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [21]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128))
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=64))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# input_dim specify the vocab count, output_dim the dense vector dim, input_length the number of words in the sentence

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.fit(X, Y, epochs = 15)

Epoch 1/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.6476 - loss: 0.5608
Epoch 2/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - accuracy: 0.8737 - loss: 0.3218
Epoch 3/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.9643 - loss: 0.1132
Epoch 4/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - accuracy: 0.9868 - loss: 0.0537
Epoch 5/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9892 - loss: 0.0369
Epoch 6/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.9932 - loss: 0.0346
Epoch 7/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - accuracy: 0.9887 - loss: 0.0543
Epoch 8/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.9951 - loss: 0.0181
Epoch 9/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - accuracy: 0.9945 - loss: 0.0301
Epoch 10/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.9960 - loss: 0.0254
Epoch 11/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - accuracy: 0.9978 - loss: 0.0157
Epoch 12/15
110/110 ━━━━━━━━━━━━━━━━━━━━ 

In [23]:

def predict(model, tokenizer, input):
  tokenized_inputs = tokenizer.texts_to_sequences(input)
  padded_inputs = pad_sequences(tokenized_inputs, padding='post')
  predictions = model.predict(padded_inputs)
  return predictions

In [24]:
accuracy = 0
for i in range(len(test_inputs)):
    print(i, ':', test_inputs[i])
    prediction = predict(model, tokenizer, [test_inputs[i]])
    print(prediction)

    if prediction > 0.5:
        predicted_label = 1
        print("Positive")
    else:
        predicted_label = 0
        print("Negative")
    if( predicted_label == test_labels[i]):
        accuracy += 1
    print(predict(model, tokenizer, [test_inputs[i]]))
accuracy = accuracy/len(test_inputs)
print(f'accuracy = {accuracy}')

0 : 2501. Optimism fuels progress and innovative thinking. 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
[[0.8687645]]
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.8687645]]
1 : 2502. An open mind leads to limitless discoveries. 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
[[0.8834195]]
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[0.8834195]]
2 : 2503. Genuine gratitude fosters enduring happiness. 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[0.82395023]]
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[0.82395023]]
3 : 2504. Resilience turns obstacles into stepping stones. 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[[0.86774254]]
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
[[0.86774254]]
4 : 2505. Generosity enhances relationships and builds communities. 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[0.8650732]]
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[[0.8650732]]
5 : 2506. Ignorance creates barriers to meaningful change. 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[0.00874539]]
Negative
1/1 ━

In [37]:
text = "not that bad this product is"
text_list = text.split()
for i in range(len(text_list)):
    if(text_list[i] not in tokenizer.word_index):
        print(text_list[i], ':not found')

prediction = predict(model, tokenizer, [text])
print(prediction)
if(prediction > 0.5):
    print("Positive")
else:
    print("Negative")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.0119201]]
Negative


In [12]:
print(len(tokenizer.word_index))


1684


In [33]:

want_to_save = int(input('want to save model? [1/0]'))
if want_to_save == 1:
    tf.keras.models.save_model(model, 'sentiment_analysis_model.keras')
    print('model saved')

want to save model? [1/0]1
model saved
